<a href="https://colab.research.google.com/github/Chanda92/CE888/blob/master/Lab5/Recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [4]:
# import data
df= pd.read_csv("https://raw.githubusercontent.com/Chanda92/CE888/master/jester-data-1.csv")
df


,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24977,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24978,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24979,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00

In [5]:
data = df.values
print(data.shape)

(24982, 101)


In [6]:
rated = np.where(data!=99)
rated[0].shape

(1835282,)

In [0]:
# helper_function ( validation set)
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(data!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(data, 0.1)

In [13]:
data[idx[0][0], idx[1][0]]

-5.78

In [14]:
# label 10% of the dataset cells as 99. 
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
#Task 2 latent factor modeling
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
# Load Data for User-ratings
data1 = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/user_ratings.csv")

In [0]:
n_latent_factors = 2

user_ratings = data1.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [21]:
latent_item_features


array([[0.953, 0.441],
       [0.147, 0.767],
       [0.899, 0.557],
       [0.652, 0.209],
       [0.95 , 0.622]])

In [22]:
latent_user_preferences

array([[0.368, 0.471],
       [0.066, 0.975],
       [0.509, 0.098],
       [0.389, 0.161],
       [0.534, 0.141],
       [0.423, 0.634],
       [0.994, 0.117],
       [0.431, 0.459],
       [0.234, 0.866],
       [0.028, 0.878]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    
def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)

In [30]:
sgd()  # Note how the MSE decreases with the number of iterations

1.3105971009927924
1.310597874079905
1.3105986447474511


In [31]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.73 ,  1.311,  4.651,  4.298,  5.162],
       [ 3.167,  1.876,  3.25 ,  7.498,  6.368],
       [ 9.   ,  1.907,  5.797,  6.635,  7.218],
       [11.02 ,  2.332,  7.095,  8.107,  8.826],
       [15.903,  1.391,  8.256,  3.244,  6.468],
       [ 2.221,  3.515,  4.488, 14.682, 11.452],
       [ 4.721,  1.166,  3.207,  4.19 ,  4.312],
       [ 6.174,  2.   ,  4.672,  7.516,  7.149],
       [ 2.549,  1.471,  2.577,  5.869,  5.002],
       [ 4.814,  0.912,  2.992,  3.086,  3.518]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data1.columns

In [34]:
comparison_data

,The Call of Cthulhu,Frankenstein,Dracula,Neuromancer,Space Odyssey
0,"(8.0, 7.73036147548541)","(2.0, 1.3113468110784028)","(nan, 4.651080812009568)","(5.0, 4.298130727902908)","(4.0, 5.161604067162342)"
1,"(3.0, 3.1665502790306146)","(2.0, 1.8762663748880772)","(nan, 3.2501581144648113)","(7.0, 7.498179753266252)","(7.0, 6.368092617608318)"
2,"(9.0, 8.999705509761078)","(nan, 1.9073059255688378)","(7.0, 5.797095938794875)","(8.0, 6.634760526037573)","(5.0, 7.218260557899971)"
3,"(nan, 11.020258203343392)","(nan, 2.3315247929258636)","(7.0, 7.094606546567473)","(8.0, 8.10722840874371)","(9.0, 8.82616018736463)"
4,"(nan, 15.903114781122488)","(1.0, 1.3911410050457325)","(8.0, 8.256024039367645)","(3.0, 3.2439595001871315)","(7.0, 6.468088534770666)"
5,"(2.0, 2.2206164938137025)","(3.0, 3.515130566460893)","(5.0, 4.488216519195993)","(nan, 14.681670800513036)","(nan, 11.452189494325435)"
6,"(4.0, 4.720681212224078)","(2.0, 1.1660797058346313)","(nan, 3.2071435484222546)","(2.0, 4.189826198306743)","(7.0, 4.3123722336478645)"
7,"(7.0, 6.173912727874322)","(1.0, 2.0002103727709093)","(2.0, 4.671679431831946)","(7.0, 7.515518827864025)","(9.0, 7.149291364550851)"
8,"(3.0, 2.548550319622112)","(3.0, 1.4714337965091466)","(nan, 2.5770209487422924)","(7.0, 5.869192520633304)","(3.0, 5.002485836525799)"
9,"(4.0, 4.81416745151009)","(nan, 0.9121170351280325)","(5.0, 2.992390268645591)","(3.0, 3.0857238368209505)","(3.0, 3.517685929710898)"
